In [1]:
import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
from tensorflow.keras.datasets import cifar10

batch_size = 512
epochs = 50
num_class = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_valid = x_train[:10000]
y_valid = y_train[:10000]

x_train = x_train[10000:]
y_train = y_train[10000:]


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.build((None, 32, 32, 3))
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(),  metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 30, 30, 32)        128       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)       

In [21]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(x_valid, y_valid), callbacks=[early_stopping])

Epoch 1/50
79/79 [==============================] - 28s 76ms/step - loss: 1.7570 - accuracy: 0.4012 - val_loss: 3.5949 - val_accuracy: 0.1032
Epoch 2/50
79/79 [==============================] - 5s 64ms/step - loss: 1.2806 - accuracy: 0.5419 - val_loss: 4.3106 - val_accuracy: 0.1032
Epoch 3/50
79/79 [==============================] - 5s 62ms/step - loss: 1.0944 - accuracy: 0.6122 - val_loss: 4.1355 - val_accuracy: 0.1091
Epoch 4/50
79/79 [==============================] - 5s 63ms/step - loss: 0.9771 - accuracy: 0.6544 - val_loss: 3.0496 - val_accuracy: 0.2538
Epoch 5/50
79/79 [==============================] - 5s 64ms/step - loss: 0.8836 - accuracy: 0.6843 - val_loss: 2.1150 - val_accuracy: 0.4047
Epoch 6/50
79/79 [==============================] - 5s 62ms/step - loss: 0.8196 - accuracy: 0.7100 - val_loss: 1.4026 - val_accuracy: 0.5493
Epoch 7/50
79/79 [==============================] - 5s 64ms/step - loss: 0.7637 - accuracy: 0.7304 - val_loss: 0.9907 - val_accuracy: 0.6621
Epoch 8/50
7

In [23]:
model.save("cnn_bn.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
pred = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {100 * pred[1]:.2f}%")

313/313 [==============================] - 2s 5ms/step - loss: 0.6007 - accuracy: 0.7971
Test Accuracy: 79.71%
